In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Sequential
import os
import matplotlib.pyplot as plt
import numpy as np 
import glob
import os
import time
from tqdm import tqdm

标准的ReesNet


In [2]:
# GPU 的问题！！
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
for name in glob.glob(data_dir + '/*'):
    print(name)
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train_label.npy


((3200, 9, 9, 204), (3200,), (50929, 9, 9, 204), (50929,))

In [4]:
train_label = tf.keras.utils.to_categorical(train_label)
test_label = tf.keras.utils.to_categorical(test_label)

In [5]:
BATCHSZ = 32
batch_size = 32
epochs =10
class_num = 16
im_height = 9
im_width = 9
im_channel = train.shape[3]
train_num = train.shape[0]
val_num = test.shape[0]

In [6]:
# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

# load train dataset
db_train = tf.data.Dataset.from_tensor_slices((train, train_label))
# train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
train_dataset = db_train.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
db_test = tf.data.Dataset.from_tensor_slices((test, test_label))
# val_dataset = val_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
val_dataset = db_test.batch(BATCHSZ).prefetch(AUTOTUNE)

In [7]:
db_train, db_test
train_dataset, val_dataset

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>)

In [8]:
class BasicBlock(layers.Layer):
    expansion = 1

    def __init__(self, out_channel, strides=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(out_channel, kernel_size=3, strides=strides,
                                   padding="SAME", use_bias=False)
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        # -----------------------------------------
        self.conv2 = layers.Conv2D(out_channel, kernel_size=3, strides=1,
                                   padding="SAME", use_bias=False)
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        # -----------------------------------------
        self.downsample = downsample
        self.relu = layers.ReLU()
        self.add = layers.Add()

    def call(self, inputs, training=False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        x = self.add([identity, x])
        x = self.relu(x)

        return x

In [9]:
class Bottleneck(layers.Layer):
    expansion = 4

    def __init__(self, out_channel, strides=1, downsample=None, **kwargs):
        super(Bottleneck, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(out_channel, kernel_size=1, use_bias=False, name="conv1")
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")
        # -----------------------------------------
        self.conv2 = layers.Conv2D(out_channel, kernel_size=3, use_bias=False,
                                   strides=strides, padding="SAME", name="conv2")
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv2/BatchNorm")
        # -----------------------------------------
        self.conv3 = layers.Conv2D(out_channel * self.expansion, kernel_size=1, use_bias=False, name="conv3")
        self.bn3 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv3/BatchNorm")
        # -----------------------------------------
        self.relu = layers.ReLU()
        self.downsample = downsample
        self.add = layers.Add()

    def call(self, inputs, training=False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        x = self.add([x, identity])
        x = self.relu(x)

        return x

In [10]:
def _make_layer(block, in_channel, channel, block_num, name, strides=1):
    downsample = None
    if strides != 1 or in_channel != channel * block.expansion:
        downsample = Sequential([
            layers.Conv2D(channel * block.expansion, kernel_size=1, strides=strides,
                          use_bias=False, name="conv1"),
            layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5, name="BatchNorm")
        ], name="shortcut")

    layers_list = []
    layers_list.append(block(channel, downsample=downsample, strides=strides, name="unit_1"))

    for index in range(1, block_num):
        layers_list.append(block(channel, name="unit_" + str(index + 1)))

    return Sequential(layers_list, name=name)


def _resnet(block, blocks_num, im_width=224, im_height=224, num_classes=1000, include_top=True):
    # tensorflow中的tensor通道排序是NHWC
    # (None, 224, 224, 3)
    # change
    input_image = layers.Input(shape=(im_height, im_width, 204), dtype="float32")
    x = layers.Conv2D(filters=64, kernel_size=7, strides=2,
                      padding="SAME", use_bias=False, name="conv1")(input_image)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(x)

    x = _make_layer(block, x.shape[-1], 64, blocks_num[0], name="block1")(x)
    x = _make_layer(block, x.shape[-1], 128, blocks_num[1], strides=2, name="block2")(x)
    x = _make_layer(block, x.shape[-1], 256, blocks_num[2], strides=2, name="block3")(x)
    x = _make_layer(block, x.shape[-1], 512, blocks_num[3], strides=2, name="block4")(x)

    if include_top:
        x = layers.GlobalAvgPool2D()(x)  # pool + flatten
        x = layers.Dense(num_classes, name="logits")(x)
        predict = layers.Softmax()(x)
    else:
        predict = x

    model = Model(inputs=input_image, outputs=predict)

    return model

In [11]:
def resnet34(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(BasicBlock, [3, 4, 6, 3], im_width, im_height, num_classes, include_top)


def resnet50(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 6, 3], im_width, im_height, num_classes, include_top)


def resnet101(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 23, 3], im_width, im_height, num_classes, include_top)

In [12]:
# 实例化模型
feature = resnet34(im_width=9, im_height=9, num_classes=class_num, include_top=False)


model = tf.keras.Sequential([feature,
                                tf.keras.layers.GlobalAvgPool2D(),
                                tf.keras.layers.Dropout(rate=0.5),
                                tf.keras.layers.Dense(1024, activation="relu"),
                                tf.keras.layers.Dropout(rate=0.5),
                                tf.keras.layers.Dense(16),
                                tf.keras.layers.Softmax()])

model.summary()

# using keras low level api for training
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1 (Functional)    (None, 1, 1, 512)         21932032  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                16400     
_________________________________________________________________
softmax (Softmax)            (None, 16)                0

In [13]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        output = model(images, training=True)
        loss = loss_object(labels, output)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, output)

@tf.function
def test_step(images, labels):
    output = model(images, training=False)
    t_loss = loss_object(labels, output)

    test_loss(t_loss)
    test_accuracy(labels, output)

In [14]:
best_test_loss = float('inf')
train_step_num = train_num // batch_size
val_step_num = val_num // batch_size
for epoch in range(1, epochs+1):
    train_loss.reset_states()        # clear history info
    train_accuracy.reset_states()    # clear history info
    test_loss.reset_states()         # clear history info
    test_accuracy.reset_states()     # clear history info

    t1 = time.perf_counter()
    for index, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        if index+1 == train_step_num:
            break
    print(time.perf_counter()-t1)

    for index, (images, labels) in enumerate(val_dataset):
        test_step(images, labels)
        if index+1 == val_step_num:
            break

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch,
                            train_loss.result(),
                            train_accuracy.result() * 100,
                            test_loss.result(),
                            test_accuracy.result() * 100))
    if test_loss.result() < best_test_loss:
        model.save_weights("./save_weights/myResNet.ckpt", save_format='tf')

6.606709599999931
Epoch 1, Loss: 4.409997463226318, Accuracy: 9.09375, Test Loss: 2.304157018661499, Test Accuracy: 27.873584747314453
2.6892331999999897
Epoch 2, Loss: 3.5891764163970947, Accuracy: 15.5625, Test Loss: 1.804103970527649, Test Accuracy: 49.717159271240234
2.6978610000001026
Epoch 3, Loss: 3.0134928226470947, Accuracy: 21.53125, Test Loss: 1.3721567392349243, Test Accuracy: 63.16192626953125
2.693465100000026
Epoch 4, Loss: 2.4246490001678467, Accuracy: 31.593748092651367, Test Loss: 1.158398985862732, Test Accuracy: 61.29399490356445
2.7028881000001093
Epoch 5, Loss: 2.0023787021636963, Accuracy: 40.40625, Test Loss: 0.861285388469696, Test Accuracy: 69.9658203125
2.7055960999999797
Epoch 6, Loss: 1.6792712211608887, Accuracy: 49.71875, Test Loss: 0.6510846018791199, Test Accuracy: 76.99952697753906
2.7018593999999894
Epoch 7, Loss: 1.455611228942871, Accuracy: 56.3125, Test Loss: 0.5890112519264221, Test Accuracy: 77.44735717773438
2.704505999999924
Epoch 8, Loss: 1.25